<div align="center">
# 🔐 Projet Mastercamp 2025  
## Surveillance & Analyse Automatisée des Vulnérabilités ANSSI (CVE)

**Participants :**  
- Abouleila Selim  
- Sozer Begumu  
- Nguedia Duval  
- Sami Samiali  

</div>


</div>


# Étape 1 : Récupération des liens des bulletins ANSSI

## Objectif
Collecter automatiquement les URLs des bulletins d’alerte et d’avis publiés par l’ANSSI pour alimenter la veille de vulnérabilités)
html = response.text


In [19]:
import feedparser

url = "https://www.cert.ssi.gouv.fr/avis/feed/"
rss_feed = feedparser.parse(url)

for entry in rss_feed.entries:
    print(entry.title, entry.link, entry.published, entry.description)

Multiples vulnérabilités dans Mattermost Server (13 mai 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0392/ Tue, 13 May 2025 00:00:00 +0000 De multiples vulnérabilités ont été découvertes dans Mattermost Server. Elles permettent à un attaquant de provoquer une atteinte à la confidentialité des données et un contournement de la politique de sécurité.
Multiples vulnérabilités dans les produits SAP (13 mai 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0396/ Tue, 13 May 2025 00:00:00 +0000 De multiples vulnérabilités ont été découvertes dans les produits SAP. Certaines d'entre elles permettent à un attaquant de provoquer une exécution de code arbitraire à distance, une atteinte à la confidentialité des données et une injection de code indirecte à distance (XSS).
Vulnérabilité dans Roundcube (02 juin 2025) https://www.cert.ssi.gouv.fr/avis/CERTFR-2025-AVI-0468/ Mon, 02 Jun 2025 00:00:00 +0000 Une vulnérabilité a été découverte dans Roundcube Roundcube Webmail. Elle perme

In [21]:
import feedparser
url2 = "https://www.cert.ssi.gouv.fr/alerte/feed"
rss_alerte = feedparser.parse(url2)

for entry in rss_alerte.entries:
    print("Title:", entry.title)
    print("Link:", entry.link)
    print("Published:", entry.published)
    print("Description:", entry.description)
    print("-" * 20) # Print a separator line

Title: Multiples vulnérabilités dans Microsoft Windows (16 septembre 2022)
Link: https://www.cert.ssi.gouv.fr/alerte/CERTFR-2022-ALE-007/
Published: Fri, 16 Sep 2022 00:00:00 +0000
Description: \[MàJ du 31 octobre 2022\] Une preuve de concept est publiquement disponible. \[Publication initiale\] Dans le cadre de son *Patch Tuesday*, en date du 13 septembre 2022, Microsoft a indiqué l'existence de multiples vulnérabilités au sein de plusieurs versions de Windows. Trois d'entre elles...
--------------------
Title: [MaJ] Multiples vulnérabilités dans Microsoft Exchange (30 septembre 2022)
Link: https://www.cert.ssi.gouv.fr/alerte/CERTFR-2022-ALE-008/
Published: Fri, 30 Sep 2022 00:00:00 +0000
Description: \[Mise à jour du 09 novembre 2022\] L'éditeur a publié un correctif (cf. section solution). En date du 29 septembre 2022, Microsoft a indiqué l'existence de deux vulnérabilités, de type zéro-jour, au sein de Windows Exchange 2013, 2016 et 2019. Ces vulnérabilités sont les suivantes : -..

# Étape 2 : Extraction des CVE

## Objectif
Extraire automatiquement les identifiants CVE (« Common Vulnerabilities and Exposures ») à partir du JSON de chaque bulletin ANSSI.

In [12]:
import requests 
import re 
url = "https://www.cert.ssi.gouv.fr/alerte/CERTFR-2024-ALE-001/json/" 
response = requests.get(url) 
data = response.json() 
#Extraction des CVE reference dans la clé cves du dict data 
ref_cves=list(data["cves"])  
#attention il s’agit d’une liste des dictionnaires avec name et url comme clés 
print( "CVE référencés ", ref_cves) 
# Extraction des CVE avec une regex 
cve_pattern = r"CVE-\d{4}-\d{4,7}" 
cve_list = list(set(re.findall(cve_pattern, str(data)))) 
print("CVE trouvés :", cve_list) 

CVE référencés  [{'name': 'CVE-2023-46805', 'url': 'https://www.cve.org/CVERecord?id=CVE-2023-46805'}, {'name': 'CVE-2024-21887', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21887'}, {'name': 'CVE-2024-21893', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21893'}, {'name': 'CVE-2024-21888', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-21888'}, {'name': 'CVE-2024-22024', 'url': 'https://www.cve.org/CVERecord?id=CVE-2024-22024'}]
CVE trouvés : ['CVE-2024-22024', 'CVE-2024-21893', 'CVE-2023-46805', 'CVE-2024-21888', 'CVE-2024-21887']


# Étape 3 : Enrichissement des CVE

## Objectif  
Pour chaque identifiant CVE extrait, récupérer automatiquement ses métadonnées (description, scores CVSS, date de publication, sources externes, EPSS…) afin de pouvoir prioriser et consolider la veille.


In [32]:
import requests

# --- Configuration ---
MITRE_API_BASE = "https://cveawg.mitre.org/api/cve"
EPSS_API_BASE  = "https://api.first.org/data/v1/epss"

# --- Fonctions d’enrichissement ---

def fetch_mitre_cve(cve_id):
    """Récupère description, score CVSS, type CWE et produits affectés via l’API MITRE."""
    result = {"cve_id": cve_id}
    try:
        resp = requests.get(f"{MITRE_API_BASE}/{cve_id}")
        resp.raise_for_status()
        data = resp.json()["containers"]["cna"]
    except Exception as e:
        result["error"] = f"Erreur MITRE: {e}"
        return result

    # Description
    descs = data.get("descriptions", [])
    result["description"] = descs[0]["value"] if descs else "Non disponible"

    # Score CVSS v3.x
    result["cvss_score"] = "Non disponible"
    for metric in data.get("metrics", []):
        if "cvssV3_1" in metric:
            result["cvss_score"] = metric["cvssV3_1"].get("baseScore", "Non disponible")
            break
        if "cvssV3_0" in metric:
            result["cvss_score"] = metric["cvssV3_0"].get("baseScore", "Non disponible")
            break

    # Type CWE
    result["cwe"] = "Non disponible"
    result["cwe_desc"] = "Non disponible"
    ptypes = data.get("problemTypes", [])
    if ptypes and "descriptions" in ptypes[0]:
        entry = ptypes[0]["descriptions"][0]
        result["cwe"] = entry.get("cweId", result["cwe"])
        result["cwe_desc"] = entry.get("description", result["cwe_desc"])

    # Produits affectés
    result["affected"] = []
    for prod in data.get("affected", []):
        vendor = prod.get("vendor", "N/A")
        name   = prod.get("product", "N/A")
        versions = [v["version"] for v in prod.get("versions", []) if v.get("status") == "affected"]
        result["affected"].append({
            "vendor": vendor,
            "product": name,
            "versions": versions or ["N/A"]
        })

    return result

def fetch_epss(cve_id):
    """Récupère le score EPSS via l’API FIRST."""
    try:
        resp = requests.get(f"{EPSS_API_BASE}?cve={cve_id}")
        resp.raise_for_status()
        data = resp.json().get("data", [])
        if data and "epss" in data[0]:
            return float(data[0]["epss"])
    except Exception:
        pass
    return "Non disponible"

def print_cve_info(rec):
    """Affiche les informations d’un CVE dans un format multi‐ligne lisible."""
    if rec.get("error"):
        print(f"\nCVE: {rec['cve_id']}\n  {rec['error']}\n" + "-"*60)
        return

    print(f"\nCVE               : {rec['cve_id']}")
    print(f"Description       : {rec['description']}")
    print(f"Score CVSS (0-10) : {rec['cvss_score']}")
    print(f"CWE               : {rec['cwe']} ({rec['cwe_desc']})")
    print(f"Score EPSS (0-1)  : {rec.get('epss_score', 'N/A')}")
    print("Produits affectés :")
    for p in rec["affected"]:
        vers = ", ".join(p["versions"])
        print(f"  • {p['vendor']} | {p['product']} | versions : {vers}")
    print("-" * 60)

# --- Exécution principale ---

if __name__ == "__main__":
    # Liste de CVE à traiter (étape 2)
    cve_list = [
        "CVE-2023-24488",
        "CVE-2023-46805",
        # ajouter d'autres CVE ici
    ]

    for cve_id in cve_list:
        rec = fetch_mitre_cve(cve_id)
        rec["epss_score"] = fetch_epss(cve_id)
        print_cve_info(rec)



CVE               : CVE-2023-24488
Description       : Cross site scripting vulnerability in Citrix ADC and Citrix Gateway  in allows and attacker to perform cross site scripting
Score CVSS (0-10) : 6.1
CWE               : CWE-79 (CWE-79 Improper Neutralization of Input During Web Page Generation ('Cross-site Scripting'))
Score EPSS (0-1)  : 0.91357
Produits affectés :
  • Citrix | Citrix ADC and Citrix Gateway  | versions : 13.1, 13.0, 12.1, 12.1-FIPS , 13.1-FIPS , 12.1-NDcPP
------------------------------------------------------------

CVE               : CVE-2023-46805
Description       : An authentication bypass vulnerability in the web component of Ivanti ICS 9.x, 22.x and Ivanti Policy Secure allows a remote attacker to access restricted resources by bypassing control checks.
Score CVSS (0-10) : 8.2
CWE               : Non disponible (Non disponible)
Score EPSS (0-1)  : 0.94398
Produits affectés :
  • Ivanti | ICS | versions : 9.1R18, 22.6R2
  • Ivanti | IPS | versions : 9.1R18,

# Étape 4 : Consolidation des Données

## Objectif  
Assembler en un seul tableau (DataFrame Pandas) l’ensemble des métadonnées ANSSI et CVE enrichies, pour pouvoir filtrer, trier et exporter facilement.

## Structure du DataFrame  
| Colonne                 | Description                                                    |
|-------------------------|----------------------------------------------------------------|
| **ID ANSSI**            | Identifiant du bulletin (ex. `CERTFR-2024-ALE-001`)            |
| **Titre ANSSI**         | Titre de l’avis ou de l’alerte                                 |
| **Type**                | `'Alerte'` ou `'Avis'`                                         |
| **Date**                | Date de publication (YYYY-MM-JJ)                               |
| **Lien**                | URL du bulletin ANSSI                                          |
| **CVE**                 | Identifiant de la vulnérabilité                                |
| **CVSS**                | Score CVSS (0–10)                                              |
| **Base Severity**       | Gravité associée (Faible, Moyenne, Élevée, Critique)          |
| **CWE**                 | Code CWE (ex. `CWE-79`)                                        |
| **EPSS**                | Score EPSS (0–1)                                               |
| **Description**         | Description détaillée issue des API                            |
| **Éditeur**             | Vendor du produit affecté                                      |
| **Produit**             | Nom du produit                                                 |
| **Versions affectées**  | Chaîne de versions impactées (séparées par des virgules)       |

## Exemple de DataFrame  
| ID ANSSI             | Titre ANSSI                         | Type   | Date       | CVE              | CVSS | Base Severity | CWE    | EPSS  | Lien                                  | Description                                   | Éditeur | Produit | Versions affectées   |
|----------------------|--------------------------------------|--------|------------|------------------|------|---------------|--------|-------|---------------------------------------|-----------------------------------------------|---------|---------|----------------------|
| CERTFR-2024-ALE-001  | Multiples vulnérabilités dans Ivanti | Alerte | 2024-01-11 | CVE-2024-49126   | 9.0  | Critique      | CWE-287| 0.85  | https://…/CERTFR-2024-ALE-001/json/   | Authentication bypass…                        | Ivanti  | ICS     | 9.1R18, 22.6R2       |
| CERTFR-2024-ALE-001  | Multiples vulnérabilités dans Ivanti | Alerte | 2024-01-11 | CVE-2023-46805   | 8.2  | Élevée        | —      | 0.94  | https://…/CERTFR-2024-ALE-001/json/   | Authentication bypass…                        | Ivanti  | IPS     | 9.1R18, 22.6R1       |

In [46]:
# Cell 1 : imports
import requests
import pandas as pd
from IPython.display import display

# Cell 2 : exemples de données
bulletins = [
    {
        "id": "CERTFR-2024-ALE-001",
        "title": "Multiples vulnérabilités dans Ivanti",
        "type": "Alerte",
        "date": "2024-01-11",
        "link": "https://www.cert.ssi.gouv.fr/alerte/CERTFR-2024-ALE-001/json/"
    }
]
cve_records = [
    {
        "cve_id": "CVE-2024-49126",
        "description": "Authentication bypass vulnerability…",
        "cvss_score": 9.0,
        "cwe": "CWE-287",
        "epss_score": 0.85,
        "affected": [
            {"vendor": "Ivanti", "product": "ICS", "versions": ["9.1R18","22.6R2"]},
            {"vendor": "Ivanti", "product": "IPS", "versions": ["9.1R18","22.6R1"]},
        ]
    },
    {
        "cve_id": "CVE-2023-46805",
        "description": "Authentication bypass vulnerability…",
        "cvss_score": 8.2,
        "cwe": None,
        "epss_score": 0.94,
        "affected": [
            {"vendor": "Ivanti", "product": "ICS", "versions": ["9.1R18","22.6R2"]},
            {"vendor": "Ivanti", "product": "IPS", "versions": ["9.1R18","22.6R1"]},
        ]
    }
]

# Cell 3 : helper pour la gravité
def get_severity_label(score):
    if score is None:
        return "Non disponible"
    if score <= 3:
        return "Faible"
    if score <= 6:
        return "Moyenne"
    if score <= 8:
        return "Élevée"
    return "Critique"

# Cell 4 : construction du DataFrame
rows = []
for b in bulletins:
    for rec in cve_records:
        for prod in rec["affected"]:
            rows.append({
                "ID ANSSI": b["id"],
                "Titre ANSSI": b["title"],
                "Type": b["type"],
                "Date": b["date"],
                "Lien": b["link"],
                "CVE": rec["cve_id"],
                "CVSS": rec["cvss_score"],
                "Base Severity": get_severity_label(rec["cvss_score"]),
                "CWE": rec["cwe"] or "Non disponible",
                "EPSS": rec["epss_score"],
                "Description": rec["description"],
                "Éditeur": prod["vendor"],
                "Produit": prod["product"],
                "Versions affectées": ", ".join(prod["versions"])
            })

df = pd.DataFrame(rows)

# Cell 5 : affichage
display(df)


,ID ANSSI,Titre ANSSI,Type,Date,Lien,CVE,CVSS,Base Severity,CWE,EPSS,Description,Éditeur,Produit,Versions affectées
0,CERTFR-2024-ALE-001,Multiples vulnérabilités dans Ivanti,Alerte,2024-01-11,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,CVE-2024-49126,9.0,Critique,CWE-287,0.85,Authentication bypass vulnerability…,Ivanti,ICS,"9.1R18, 22.6R2"
1,CERTFR-2024-ALE-001,Multiples vulnérabilités dans Ivanti,Alerte,2024-01-11,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,CVE-2024-49126,9.0,Critique,CWE-287,0.85,Authentication bypass vulnerability…,Ivanti,IPS,"9.1R18, 22.6R1"
2,CERTFR-2024-ALE-001,Multiples vulnérabilités dans Ivanti,Alerte,2024-01-11,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,CVE-2023-46805,8.2,Critique,Non disponible,0.94,Authentication bypass vulnerability…,Ivanti,ICS,"9.1R18, 22.6R2"
3,CERTFR-2024-ALE-001,Multiples vulnérabilités dans Ivanti,Alerte,2024-01-11,https://www.cert.ssi.gouv.fr/alerte/CERTFR-202...,CVE-2023-46805,8.2,Critique,Non disponible,0.94,Authentication bypass vulnerability…,Ivanti,IPS,"9.1R18, 22.6R1"
